## Fake News Classifier Using Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('train/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
###Drop Nan Values
df=df.dropna()


In [ ]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [ ]:
## Get the Dependent features
y=df['label']

In [ ]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.2.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

### Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4943 1356 4795]
 [   0    0    0 ... 3508  617  346]
 [   0    0    0 ... 4467 2701 1119]
 ...
 [   0    0    0 ... 4823 4459 4752]
 [   0    0    0 ... 3546 1870  906]
 [   0    0    0 ... 3276 1179 2262]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4220,
       3165, 1585, 3063,  201, 4710, 4782, 4943, 1356, 4795])

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [ ]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 35ms/step - loss: 0.3051 - accuracy: 0.8561 - val_loss: 0.2011 - val_accuracy: 0.9167
Epoch 2/10
192/192 [==============================] - 5s 26ms/step - loss: 0.1404 - accuracy: 0.9451 - val_loss: 0.2111 - val_accuracy: 0.9180
Epoch 3/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0987 - accuracy: 0.9635 - val_loss: 0.2341 - val_accuracy: 0.9171
Epoch 4/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0755 - accuracy: 0.9731 - val_loss: 0.2541 - val_accuracy: 0.9095
Epoch 5/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0536 - accuracy: 0.9816 - val_loss: 0.3352 - val_accuracy: 0.9147
Epoch 6/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0392 - accuracy: 0.9869 - val_loss: 0.3757 - val_accuracy: 0.9125
Epoch 7/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0323 - accuracy: 0.9895 - val_loss: 0.3839 - val_accuracy: 0.9082

### Performance Metrics And Accuracy

In [ ]:

y_pred1=model1.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred1)

array([[3093,  326],
       [ 238, 2378]], dtype=int64)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9065451532725767

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      3419
           1       0.88      0.91      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

